In [2]:
!pip install opencv-python

In [24]:
import cv2
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import Image,Video
import numpy as np


In [4]:
net=cv2.dnn.readNet('yolov3.weights','yolov3.cfg')

layer_names=net.getLayerNames()
out_layers_index=net.getUnconnectedOutLayers()
out_layers=[layer_names[index[0]-1] for index in out_layers_index]

with open('coco.names') as f:
    classes=f.read().split('\n')
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', '']


In [5]:
def draw_obj(img,index,score,box):
    x,y,w,h=box
    start=(x,y)
    end=(x+w,y+h)
    color=(0,255,0)
    width=2
    img=cv2.rectangle(img,start,end,color,width)
    
    start=(x-10,y-10)
    font_size=1
    font = cv2.FONT_HERSHEY_SIMPLEX 
    color = (0, 255, 0) 
    width=2  
    text=classes[index]
    img = cv2.putText(img, text, start, font, font_size, color, width, cv2.LINE_AA) 
    return img

In [6]:
def apply_yolo(img):
    h,w,d=img.shape
    blob=cv2.dnn.blobFromImage(img, 1/255, (608,608), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    outs=net.forward(out_layers)
    
    class_indexes=[]
    class_scores=[]
    boxes=[]

    import numpy as np
    for out in outs:
        for obj in out:
            scores=obj[5:]
            class_index=np.argmax(scores)
            class_score=scores[class_index]
            if class_score >0:
                centr_x=int(obj[0]*w)
                centr_y=int(obj[1]*h) #получаем координаты начала объекта 
                obj_w=int(obj[2]*w)#ширина объекта
                obj_h=int(obj[3]*h)#высота объекта
                x=centr_x-obj_w//2
                y=centr_y-obj_h//2
                box=[x,y,obj_w,obj_h]
                boxes.append(box)
                class_indexes.append(class_index)
                class_scores.append(float(class_score))
        chosen_boxes=cv2.dnn.NMSBoxes(boxes,class_scores, 0,0.4)
       
    for box_index in chosen_boxes:
        box_index=box_index[0]
        img=draw_obj(img,class_indexes[box_index],class_scores[box_index],boxes[box_index])
    return img

In [ ]:
#Можно воспользоваться библиотекой matplotlib вместо создания нового видео-файла, но страдает быстродействие, 
#cap=cv2.VideoCapture('vudeo.mp4')
#while cap.isOpened():
#    ret, frame=cap.read()
#    if not ret:
#        break
#    frame=apply_yolo(frame)
#    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#    frame=cv2.resize(frame,(2560//2, 1400//2))
#    plt.figure(figsize=(20,15))
#    plt.imshow(frame)
#    display.clear_output(wait=True)
#    display.display(plt.gcf())
#cap.release()


In [36]:
size=(2560//4, 1400//4)
codec=cv2.VideoWriter_fourcc(*'H264')
out=cv2.VideoWriter('output.mp4', codec, 5, size)
#codec=cv2.VideoWriter_fourcc(*'MJPG')
#out=cv2.VideoWriter('output.avi', codec, 5, size)

cap=cv2.VideoCapture('video.mp4')
counter=0
while cap.isOpened() and counter<20:
    counter+=1
    print('.',end='')
    ret, frame=cap.read()
    if not ret:
        break
    frame=apply_yolo(frame)
    frame=cv2.resize(frame,(2560//4, 1400//4))
    
    out.write(frame)

cap.release()
out.release()

....................

In [37]:
Video('output.mp4')